In [17]:
# ปํญหาที่เจอ

# 1. ทำ one hot encoding ไม่ได้ (tags)
# 2. เกมใน Steam มีเยอะมาก ประมาณ 50K+ ถ้าเอาหมดต้องดึง api ทีละเกม ใช้เกมละ 1 sec min
# 3. 

# to do list

# 1. ตัด field ที่ไม่ใช้ออก
# 2. สร้างตารางที่มี tag ทั้งหมด ค่าเป็น 0 ทั้งหมด -> เรียง tag ของแต่ละเกมตามตารางรวม -> เติมข้อมูลของแต่ละเกม -> แก้ col ของ tag ที่มีเป็น 1
# 3. Association rule
# 4. เก็บ Landing data (ข้อมูลดิบจาก api)

In [18]:
from urllib import request
import requests
import json
import csv
import pandas as pd
import numpy as np
import time
import ast

In [19]:
# เลขหน้าที่ต้องการ (0 - 57)
page_global = int(input())
print('Page =' ,page_global)

Page = 9


In [20]:
# ดึงข้อมูลของหน้า เพื่อหา appid
def r_page(page = page_global):
    n_url = 'https://steamspy.com/api.php?request=all&page=' + str(page)
    raw = requests.get(n_url)
    r_json = raw.json()

    return r_json

# สร้าง csv ของหน้า
def page_csv(page = page_global):
    f_name = "r_page_" + str(page) + ".csv"
    with open(f_name, "w", encoding="utf-8") as file :
        r_json = r_page(page)
        games_id = []
        for key in r_json.keys():
            games_id.append(key)
        feildnames = r_json[games_id[0]].keys()
        writer = csv.DictWriter(file, fieldnames = feildnames)
        writer.writeheader()
        for gameid in r_json.keys():
            writer.writerow(r_json[gameid])

    return f_name

r_page_csv = page_csv()
print("สร้าง file %s สำเร็จ" % r_page_csv)
# f_name = "page1.csv"
# d_csv = pd.read_csv("page1.csv")
# print(d_csv.head(3))


สร้าง file r_page_9.csv สำเร็จ


In [21]:
r_csv = pd.read_csv(r_page_csv)
r_csv = r_csv.drop("score_rank", axis=1)
r_csv.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   appid            1000 non-null   int64 
 1   name             1000 non-null   object
 2   developer        996 non-null    object
 3   publisher        997 non-null    object
 4   positive         1000 non-null   int64 
 5   negative         1000 non-null   int64 
 6   userscore        1000 non-null   int64 
 7   owners           1000 non-null   object
 8   average_forever  1000 non-null   int64 
 9   average_2weeks   1000 non-null   int64 
 10  median_forever   1000 non-null   int64 
 11  median_2weeks    1000 non-null   int64 
 12  price            1000 non-null   int64 
 13  initialprice     1000 non-null   int64 
 14  discount         1000 non-null   int64 
 15  ccu              1000 non-null   int64 
dtypes: int64(12), object(4)
memory usage: 125.1+ KB


In [22]:
# ดึง appid ของหน้า
r_csv = pd.read_csv(r_page_csv)
games_id = r_csv['appid']

# ดึง detail ของเกมด้วย appid
def r_details(count = len(games_id), id = games_id):
    
    # กำหนดจำนวน data point ที่ต้องการ (สำหรับการทดสอบ)
    sample_id = id[:count]

    # api detail
    url = 'https://steamspy.com/api.php?request=appdetails&appid='
    result_list = []
    for id in sample_id:
        url_id = url + str(id)
        r_id = requests.get(url_id)
        result_list.append(r_id.json())
        time.sleep(1)

    return result_list


# field = appid, name, developer, publisher, score_rank, positive, negative, userscore, owners, average_forever,
#  average_2weeks, median_forever, median_2weeks, price, initialprice, discount, ccu, languages, genre, tags

# rd_json = raw.json()
# rd_str = json.dumps(rd_json, indent=2)
# print(rd_str)

In [23]:
# ใส่จำนวนที่อยากรัน
results_list = r_details()

def csv_tag( result_list = results_list): # , tags_list = tags_list
    
    f_name = "r_tag_page_" + str(page_global) + ".csv"

    # เก็บ keys ของ tags
    # for n in range(len(result_list)):
    #     (result_list[n])['tags'] = tags_list[n]

    # สร้าง csv
    with open(f_name, "w", encoding="utf-8") as file :
        csv_file = csv.writer(file)
        csv_file.writerow((result_list[0]).keys())
        for game in result_list:
            csv_file.writerow(game.values())
        
    return f_name

r_tag_f_name = csv_tag()
print("สร้าง file: %s สำเร็จ" % r_tag_f_name)

สร้าง file: r_tag_page_9.csv สำเร็จ


In [24]:
d = pd.read_csv('r_tag_page_' + str(page_global) + '.csv')
d.tail()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags
995,1276660,Tinkertown,Headup,Headup,NaN,619,201,0,"50,000 .. 100,000",215,0,270,0,849,1699,50,58,"English, French, German, Simplified Chinese, J...","Adventure, Indie, RPG, Early Access","{'Adventure': 95, 'Indie': 86, 'RPG': 84, 'Pix..."
996,458030,StarFringe: Adversus,"Casualogic, Droid Riot",Trinity Project,NaN,31,32,0,"50,000 .. 100,000",0,0,0,0,99,99,0,3,"English, Russian","Strategy, Early Access","{'Strategy': 106, 'RTS': 99, 'Base-Building': ..."
997,549860,One Night Stand,Kinmoku,Kinmoku,NaN,556,67,0,"50,000 .. 100,000",177,0,231,0,299,299,0,0,"English, Russian, French, Portuguese - Brazil,...","Adventure, Casual, Indie, Simulation","{'Indie': 105, 'Visual Novel': 99, 'Choices Ma..."
998,503480,Mahluk:Dark Demon,Serkan Bakar,Serkan Bakar,NaN,94,51,0,"50,000 .. 100,000",126,0,140,0,399,399,0,0,English,"Action, Adventure, Casual, Indie, RPG","{'Action': 151, 'Adventure': 145, 'Indie': 142..."
999,1005410,Freedom Finger,"Wide Right Interactive, Mark Zorn",Wide Right Interactive,NaN,112,7,0,"50,000 .. 100,000",35,0,35,0,799,799,0,1,"English, Russian, Simplified Chinese, German, ...","Action, Indie","{'Bullet Hell': 633, ""Shoot 'Em Up"": 629, 'Mus..."
